In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing Libaries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20,10)
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
import pickle
import json

Reading File

We will use pandas read_csv function to read the data in csv file.

In [ ]:
filepath = '../input/bengaluru-house-price-data/Bengaluru_House_Data.csv'
data = pd.read_csv(filepath)
data.head(20)

Data Preprcessing

Data preprocessing is a data mining technique that involves transforming raw data into an understandable format.

In [ ]:
data.shape

In [ ]:
data.groupby('area_type')['area_type'].agg('count')

groupby() function is used to split the data into groups based on some criteria and agg() function abbreviation of aggregate is used to define what we want to do with the grouped data.

dropping useless columns

In [ ]:
data2 = data.drop(['area_type','society','balcony','availability'], axis='columns')
data2.head()

finding total empty values in each column

In [ ]:
data2.isnull().sum() 

dropping 'NA' values

In [ ]:
data3 = data2.dropna()  
data3.isnull().sum()

In [ ]:
data3.shape

In [ ]:
data3['size'].unique() # checking unique values in size column

In [ ]:
data3['bhk'] = data3['size'].apply(lambda x: int(x.split(' ')[0])) # getting the number of bedrooms 
#from size column

In [ ]:
data3.head()

In [ ]:
data3['bhk'].unique()

In [ ]:
data3[data3.bhk>20] 

Finding rows with extraordinary values (outliers)


In [ ]:
data3.total_sqft.unique()

Function to convert values into float dtype.

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
data3[~data3['total_sqft'].apply(is_float)].head(20) # finding values those not got converted

 Function to convert those unusual format of data

In [ ]:
def convert_sqft_to_sum(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1])) / 2
    try:
        return float(x)
    except:
        return None

In [ ]:
convert_sqft_to_sum('2100 - 2850') # usage of the function

In [ ]:
data4 = data3.copy()
data4['total_sqft'] = data4['total_sqft'].apply(convert_sqft_to_sum)
data4.head()

In [ ]:
data4.loc[30] # loc function is used to see data row-wise

In [ ]:
data5 = data4.copy()   # copy function is used to copy the whole dataframe
data5.head()

Making a new column in the dataframe named `price_per_sqft` and see the logic to create it

In [ ]:
data5['price_per_sqft'] = data5['price'] * 100000/data5['total_sqft']
data5.head()

In [ ]:
len(data5.location.unique())

In [ ]:
data5.location = data5.location.apply(lambda x : x.strip()) # strip is used to remove the white spaces around the data points
location_stats = data5.groupby('location')['location'].agg('count').sort_values(ascending=False) # sorting the location column in descending order
location_stats

In [ ]:
len(location_stats[location_stats<=10]) # totaling the minor locations

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
data5.location = data5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x) # changing minor locations into `other`

In [ ]:
len(data5.location.unique())

In [ ]:
data5[data5.total_sqft / data5.bhk < 300].head() # checking for outliners; like a house with 1407 sq. area can't have 6 bedrooms

In [ ]:
data5.shape

In [ ]:
data6 = data5[~(data5.total_sqft/data5.bhk<300)]

In [ ]:
data6.price_per_sqft.describe()

In [ ]:
data6.shape

Data Cleaning

In this section, we will find outliers and try to remove them.

Function to clear stuff (outliers) in the price_per_sqft column so that we don't live in a hypothetical dataset

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

In [ ]:
data7 = remove_pps_outliers(data6)
data7.shape

In [ ]:
data7.head()

Function to clear stuff (outliers) in the bhk column so that we don't live in a hypothetical dataset.

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

data8 = remove_bhk_outliers(data7)
data8.shape

Data Visualization

In [ ]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (15, 10)
    plt.scatter(bhk2.total_sqft, bhk2.price,color='blue', label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft, bhk3.price,marker='+',color='green', label='3 BHK', s=50)
    plt.xlabel('Total Square Feet Area')
    plt.ylabel('Price Per Square Feet')
    plt.title(location)
    plt.legend()

In [ ]:
plot_scatter_chart(data7, 'Whitefield')

In [ ]:
plot_scatter_chart(data7,"Hebbal")

In [ ]:
plt.hist(data8.price_per_sqft, rwidth=0.8)    # visualization the price_per_sqft column
plt.xlabel('Price per Square Feet')
plt.ylabel('Count')

In [ ]:
data8.bath.unique()

In [ ]:
data8[data8.bath>10] # Again outliers

In [ ]:
plt.hist(data8.bath, rwidth=0.8)
plt.xlabel('Number of bathrooms')
plt.ylabel('Count')

In [ ]:
data8[data8.bath > data8.bhk + 2] # more bathrooms that bedrooms it's a outliers

In [ ]:
data9 = data8[data8.bath < data8.bhk + 2]
data9.shape

In [ ]:
data9.sample()    # sample return a random row from the dataset

In [ ]:
data10 = data9.drop(['size',"price_per_sqft"],axis='columns')    # removing or dropping 'size' and 'prize_per_sqft' as we don't require them any more
data10.head()

Creating Dummies

We will use pandas' get_dummies() to create dummies variables. It is used for data manipulation. It converts categorical data into dummy or indicator variables.

In [ ]:
dummies = pd.get_dummies(data10.location)     
dummies.head()

In [ ]:
data11 = pd.concat([data10, dummies.drop('other', axis='columns')], axis='columns')    # joining the dummy values again with the dataset except 'other' column
data11.head()

In [ ]:
data12 = data11.drop('location', axis='columns')    # dropping original location as now we have dummmies in its place.
data12.head()

Splitting Data for Training and Testing

Before Training the model, it is required to split the data into train and test data. For this we will use, sklearn's train_test_split

In [ ]:
X = data12.drop('price', axis='columns')    # dropping price column as we don't want it in our train dataset
X.head()

In [ ]:
X.shape

In [ ]:
y = data12.price    # taking the price column as our target to predict

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)    # splitting the the data into train and test data as 80 : 20 ratio

In [ ]:
X_train.shape

Training the Models


we will be trying more than one model

In [ ]:
lr_clf = LinearRegression()     # first trying training with LinearRegression 
lr_clf.fit(X_train, y_train)
lr_clf.score(X_test, y_test)

In [ ]:
cv = ShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)     # ShuffleSplit is just a another type of splitting data
cross_val_score(LinearRegression(), X, y, cv=cv)  

Cross Validation is mainly used for the comparison of different models. For each model, we may get the average generalization error on the k validation sets. Then we will be able to choose the model with the lowest average generation error as our optimal model.

In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression':{
            'model': LinearRegression(),
            'params':{
                'normalize':[True,False]
            }
        },
        'lasso':{
            'model': Lasso(),
            'params':{
                'alpha' : [1,2],
                'selection':['random','cyclic']
            }
        },
        'decision_tree':{
            'model': DecisionTreeRegressor(),
            'params':{
                'criterion':['mse','friedman_mse'],
                'splitter':['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'],config['params'], cv=cv, return_train_score=False)     # GridSearchCV is the main focus as it helps to try out the different parameters for the different models.
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score':gs.best_score_,
            'best_params':gs.best_params_
        })
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])     # At last binding the results of the models with best params. into a DataFrame.

In [ ]:
find_best_model_using_gridsearchcv(X, y)

As we can see the LinearRegression model performed the best, so we are going to use as it for prediction.

Predicting the prices using LinearRegression in Lakhs.

In [ ]:
def predict_price(location,sqft,bath,bhk):
    loc_index = np.where(X.columns==location)[0][0]     # np.where() function returns the indices of elements in an input array where the given condition is satisfied.
    
    x = np.zeros(len(X.columns))    # np.zeros() function returns a new array of given shape and type, with zeros.
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1
        
    return lr_clf.predict([x])[0]

In [ ]:
print(predict_price('1st Phase JP Nagar', 1000, 2, 3).round(3),'Lakhs')

 Saving Model

In [ ]:
with open('BHP_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

In [ ]:
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open('columns.json','w') as f:
    f.write(json.dumps(columns))